In [7]:
from sympy import *
from sympy import sin, pde_separate
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector.printing import vlatex, vpprint
import numpy as np

import matplotlib.pyplot as plt
init_printing(use_latex='mathjax',use_unicode=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [8]:
q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1 q2 q3 q4 q5 q6')
dq1, dq2, dq3, dq4, dq5, dq6 = symbols('dq1 dq2 dq3 dq4 dq5 dq6')
ddq1, ddq2, ddq3, ddq4, ddq5, ddq6 = symbols('ddq1 ddq2 ddq3 ddq4 ddq5 ddq6')
Q1, Q2, Q3, Q4, Q5, Q6 = symbols('Q1 Q2 Q3 Q4 Q5 Q6')
m, Ixx, Iyy, Izz, g = symbols('m Ixx Iyy Izz g ', constant = True)

q_3, q_5, q_6 = symbols('q_3 q_5 q_6')

In [9]:
q = Matrix([q1, q2, q3, q4, q5, q6])
dq = diff(q, Symbol('t'))
ddq = diff(dq, Symbol('t'))


$
T(q)=\frac{1}{2}\dot{q}^{T}M(q)*\dot{q}
$

where M(q) is defined as:

$
M(q)=J_{v}(q)^{T}m_{rr}J_{v}(q)+J_{w}(q)^{T}m_{\theta\theta}J_{w}(q)
$

In [10]:
J_w = Matrix([[0,0,0,-sin(q[4]), 0, 1],
             [0,0,0, cos(q[4])*sin(q[5]), cos(q[5]), 0],
             [0,0,0, cos(q[4])*cos(q[5]), sin(q[5]), 0]])
J_v = Matrix([[1,0,0,0,0,0],
             [0,1,0,0,0,0],
             [0,0,1,0,0,0]])
m_rr = eye(3)*m
m_tt = Matrix([[Ixx, 0, 0],
              [0, Iyy, 0],
              [0, 0, Izz]])
M_q = J_v.T * m_rr * J_v + J_w.T * m_tt * J_w
M_q = trigsimp(M_q)

T = trigsimp(simplify(trigsimp(0.5*dq.T*M_q*dq)))

V = -m*g*q[2]

L = T[0]-V
L = collect(L, [dq[3]**2, dq[4]**2])

L = L.trigsimp(method = 'fu')
L

                   2                                2                    2    
        ⎛d        ⎞                      ⎛d        ⎞          ⎛d        ⎞     
0.5⋅Ixx⋅⎜──(q₆(t))⎟  + g⋅m⋅q₃(t) + 0.5⋅m⋅⎜──(q₁(t))⎟  + 0.5⋅m⋅⎜──(q₂(t))⎟  + 0
        ⎝dt       ⎠                      ⎝dt       ⎠          ⎝dt       ⎠     

                2                                                             
     ⎛d        ⎞    ⎛           2                     2                 ⎞ ⎛d  
.5⋅m⋅⎜──(q₃(t))⎟  + ⎝0.5⋅Iyy⋅cos (q₆(t)) - 0.5⋅Izz⋅cos (q₆(t)) + 0.5⋅Izz⎠⋅⎜──(
     ⎝dt       ⎠                                                          ⎝dt 

       2                                                                      
      ⎞    ⎛                     d                                            
q₅(t))⎟  + ⎜- 1.0⋅Ixx⋅sin(q₅(t))⋅──(q₆(t)) + 0.25⋅Iyy⋅(-sin(q₅(t) - 2⋅q₆(t)) +
      ⎠    ⎝                     dt                                           

                                                 

The L above is with no trig simplification, actually it is expanded.

In the next cell you are looking at L but simplified with trig by hand.

In [11]:
L = (0.5*m*dq[0]**2 + 
     0.5*m*dq[1]**2 + 
     0.5*m*dq[2]**2 + 
     (0.5*Ixx*sin(q[4])**2 + 0.5*Iyy*cos(q[5])**2 - 0.5*(Iyy-Izz)*(cos(q[4])**2)*cos(q[5])**2)*dq[3]**2 +
     0.5*((Iyy - Izz)*cos(q[5])**2 + Izz)*dq[4]**2 + 
     0.5*Ixx*dq[5]**2 +
     (Iyy + Izz)*cos(q[4])*sin(q[5])*cos(q[5])*dq[3]*dq[4] - 
     Ixx*sin(q[4])*dq[3]*dq[5] +
     m*g*q[2]
    )

L

                                                          2                   
                 d         d                   ⎛d        ⎞                    
- Ixx⋅sin(q₅(t))⋅──(q₄(t))⋅──(q₆(t)) + 0.5⋅Ixx⋅⎜──(q₆(t))⎟  + g⋅m⋅q₃(t) + 0.5⋅
                 dt        dt                  ⎝dt       ⎠                    

             2                    2                    2                      
  ⎛d        ⎞          ⎛d        ⎞          ⎛d        ⎞                       
m⋅⎜──(q₁(t))⎟  + 0.5⋅m⋅⎜──(q₂(t))⎟  + 0.5⋅m⋅⎜──(q₃(t))⎟  + (Iyy + Izz)⋅sin(q₆(
  ⎝dt       ⎠          ⎝dt       ⎠          ⎝dt       ⎠                       

                                                                              
                          d         d           ⎛                             
t))⋅cos(q₅(t))⋅cos(q₆(t))⋅──(q₄(t))⋅──(q₅(t)) + ⎝0.5⋅Izz + 0.5⋅(Iyy - Izz)⋅cos
                          dt        dt                                        

                     2                           

Euler-Lagrange equations of motions from the following equation:

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{j}}})-\frac{\partial\mathcal{L}(q)}{\partial q_{j}}=Q_{j}(q)
$


In [12]:
# d/dt(dL/d(dq)) - dL/dq = Q
dl1 = diff(diff(L, dq[0]), Symbol('t'))
dl2 = diff(diff(L, dq[1]), Symbol('t'))
dl3 = diff(diff(L, dq[2]), Symbol('t'))
dl4 = diff(diff(L, dq[3]), Symbol('t'))
dl5 = diff(diff(L, dq[4]), Symbol('t'))
dl6 = diff(diff(L, dq[5]), Symbol('t'))

l1 = diff(L, q[0])
l2 = diff(L, q[1])
l3 = diff(L, q[2])
l4 = diff(L, q[3])
l5 = diff(L, q[4])
l6 = diff(L, q[5])

EL1 = dl1 - l1 - Q1
EL2 = dl2 - l2 - Q2
EL3 = dl3 - l3 - Q3
EL4 = dl4 - l4 - Q4
EL5 = dl5 - l5 - Q5
EL6 = dl6 - l6 - Q6

EL6.coeff(ddq[5])

eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
EL4

1.0⋅Ixx

                   2                                                          
                  d                          d         d                      
- Ixx⋅sin(q₅(t))⋅───(q₆(t)) - Ixx⋅cos(q₅(t))⋅──(q₅(t))⋅──(q₆(t)) - Q₄ - (Iyy +
                   2                         dt        dt                     
                 dt                                                           

                                                  2                           
                                       ⎛d        ⎞                   2        
 Izz)⋅sin(q₅(t))⋅sin(q₆(t))⋅cos(q₆(t))⋅⎜──(q₅(t))⎟  - (Iyy + Izz)⋅sin (q₆(t))⋅
                                       ⎝dt       ⎠                            
                                                                              

                                                                              
           d         d                                                        
cos(q₅(t))⋅──(q₅(t))⋅──(q₆(t)) + (Iyy + Izz)⋅sin(q

In [13]:
#After hand simplifying this is rewritten 

EL4 = (-(0.5*Iyy - 0.5*Izz)*(cos(q[4])**2)*(cos(q[5])**2)*ddq[3] +
       (Iyy + Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[4] -
       Ixx*sin(q[4])*ddq[5] +
       2*sin(q[4])*cos(q[4])*(Ixx + (Iyy - Izz)*cos(q[5])**2)*dq[3]*dq[4] +
       sin(q[5])*cos(q[5])*((Iyy-Izz)*cos(q[4])**2 - Ixx)*dq[3]*dq[5] -
       Q4)

EL5 = ((Iyy + Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[3] +
       (Izz + (Iyy - Izz)*cos(q[5])**2)*ddq[4] +
       sin(q[4])*cos(q[4])*(Ixx + (Iyy - Izz)*cos(q[5])**2)*dq[3]**2 -
       2*(Iyy - Izz)*sin(q[5])*cos(q[5])*dq[4]*dq[5] +
       cos(q[4])*(Ixx +(Iyy + Izz)*cos(2*q[5]))*dq[3]*dq[5] -
       Q5)

EL6 = (-Ixx*sin(q[4])*ddq[3] +
       Ixx*ddq[5] -
       sin(q[5])*cos(q[5])*((Iyy-Izz)*cos(q[4])**2 + Iyy)*dq[3]**2 +
       (Iyy-Izz)*sin(q[5])*cos(q[5])*dq[4]**2 -
       cos(q[4])*(Ixx + (Iyy + Izz)*cos(2*q[5]))*dq[3]*dq[4] -
       Q6)



eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
print(vlatex(EL1))

- Q_{1} + 1.0 m \ddot{q}_{1}


In [14]:
#This is the main function to seperate the double derivative

F = solve(eqs, variables)

In [15]:
#This breaks out the 6 equations all f(Q)

F1 = F[ddq[0]]
F2 = F[ddq[1]]
F3 = F[ddq[2]]
F4 = F[ddq[3]]
F5 = F[ddq[4]]
F6 = F[ddq[5]]

# print(vlatex(F3))
F6


                      ⎛                                  2                    
                      ⎜                       ⎛d        ⎞                   d 
- 2.0⋅Ixx⋅(Iyy + Izz)⋅⎜0.5⋅Ixx⋅sin(2.0⋅q₅(t))⋅⎜──(q₄(t))⎟  + Ixx⋅cos(q₅(t))⋅──
                      ⎝                       ⎝dt       ⎠                   dt
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                     2        
        d                                        2        ⎛d        ⎞         
(q₄(t))⋅──(q₆(t)) + Iyy⋅sin(q₅(t))⋅cos(q₅(t))⋅cos (q₆(t))⋅⎜──(q₄(t))⎟  - Iyy⋅s
        dt                                                ⎝dt       ⎠         
──────────────────────────────────────────────────────────────────────────────
                                                   

In [16]:
G = zeros(12,12)

#Dummy lists for the loop
_f = [F1,F2,F3,F4,F5,F6]
_q = [Q1,Q2,Q3,Q4,Q5,Q6]


for n in range(0,6):
    
    G[6+n,6+n] = apart(_f[n],_q[n]).args[1]

G[9,10] = apart(F4,Q5).args[1]
G[9,11] = apart(F4,Q6).args[1]

G[10,9] = apart(F5,Q4).args[1]
G[10,11] = apart(F5,Q6).args[1]

G[11,9] = apart(F6,Q4).args[1]
G[11,10] = apart(F6,Q5).args[1]

    

In [17]:
#Workup G from the "apart" function and args[1]

G

⎡0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢0  0  0  0  0  0  0  0  0                                                    
⎢                                                                             
⎢                  1                                

In [18]:
a = sympify(G[11,11])
b = sympify(F6)

# Add(b,-a).doit()
# sympify(b-a, evaluate = False).doit()
# sympify(a,b,evaluate = False)

In [53]:
def funcs(t,z):
    q1, q2, q3, q4, q5, q6 = z
    F1.subs(q[4],q4)
    return [F1, F2, F3, F4, F5, F6]

In [54]:
F4.subs(Izz,1)
# F4.subs([Ixx,Iyy,Izz],[1,1,1])

 ⎛              ⎛                                  2                          
 ⎜              ⎜                       ⎛d        ⎞                   d       
-⎜2.0⋅(Iyy + 1)⋅⎜0.5⋅Ixx⋅sin(2.0⋅q₅(t))⋅⎜──(q₄(t))⎟  + Ixx⋅cos(q₅(t))⋅──(q₄(t)
 ⎝              ⎝                       ⎝dt       ⎠                   dt      
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                               2              
  d                                        2        ⎛d        ⎞               
)⋅──(q₆(t)) + Iyy⋅sin(q₅(t))⋅cos(q₅(t))⋅cos (q₆(t))⋅⎜──(q₄(t))⎟  - Iyy⋅sin(2.0
  dt                                                ⎝dt       ⎠               
──────────────────────────────────────────────────────────────────────────────
                                                   

In [56]:
help(diff)

Help on function diff in module sympy.core.function:

diff(f, *symbols, **kwargs)
    Differentiate f with respect to symbols.
    
    This is just a wrapper to unify .diff() and the Derivative class; its
    interface is similar to that of integrate().  You can use the same
    shortcuts for multiple variables as with Derivative.  For example,
    diff(f(x), x, x, x) and diff(f(x), x, 3) both return the third derivative
    of f(x).
    
    You can pass evaluate=False to get an unevaluated Derivative class.  Note
    that if there are 0 symbols (such as diff(f(x), x, 0), then the result will
    be the function (the zeroth derivative), even if evaluate=False.
    
    Examples
    
    >>> from sympy import sin, cos, Function, diff
    >>> from sympy.abc import x, y
    >>> f = Function('f')
    
    >>> diff(sin(x), x)
    cos(x)
    >>> diff(f(x), x, x, x)
    Derivative(f(x), (x, 3))
    >>> diff(f(x), x, 3)
    Derivative(f(x), (x, 3))
    >>> diff(sin(x)*cos(y), x, 2, y, 2)
   